# 과제: 네이버 영화 정보 및 평점 크롤링

- 대상: 예매순 상위 5개의 현재 상영 중인 영화
- 수집할 항목: 영화 제목, 주연배우 3인, 네티즌 평점, 관람객 평점, 기자/평론가 평점, 관람객 별점 리뷰 20건 공감순으로(평점, 작성자닉네임, 리뷰본문)

In [1]:
import requests
from bs4 import BeautifulSoup
import re

In [2]:
import pandas as pd

### 1. 예매순 상위 5개의 현재 상영 중인 영화 가져오기

영화 제목, 주연배우 3인

In [3]:
def movie_title_url_actor(movienum, actornum):
    url = 'https://movie.naver.com/movie/running/current.nhn'
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    
    movie = soup.select("dl.lst_dsc")
    
    titlelist = []
    actorlist = []

    # 영화 제목
    for m in movie: 
        title = m.select_one("dt.tit a")

        titlelist.append(title.text) # 텍스트로 변환해 title 리스트에 넣는다 
        if len(titlelist) == movienum:# 원하는 영화 개수만큼 추출하기로 한다 
            break

    # 주연배우
    for m in movie:         
        actors = m.select("dl.info_txt1 dd:nth-of-type(3) a") 
        # info_txt1에는 a로 시작하는 항목이 여러 개가 있다 
        # 이 중 배우에 해당하는 항목은 3번째 하위항목이므로, 이 때의 값을 불러온다    

        actor = []
        for a in actors: 
            actor.append(a.text) # 텍스트로 변환해 actor 리스트에 넣는다 
            if len(actor) == actornum: # 원하는 주연배우 수 만큼 추출하기로 한다 
                break
        actorlist.append(actor)
        if len(actorlist) == len(titlelist): # 영화 개수에 맞춰서 배우 리스트를 추출한다 
            break
        
    return titlelist, actorlist


### 2. 해당 영화의 평점 가져오기

네티즌 평점, 관람객 평점, 기자/평론가 평점

In [4]:
def get_grade(movienum):
    url = 'https://movie.naver.com/movie/running/current.nhn'
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')    
    
    movie = soup.select("dl.lst_dsc")
    
    html_list = []

    for m in movie: 
        title = m.select_one("dt.tit a")
        url = title.attrs["href"] # 영화마다의 고유 url들을 불러온다 
        each_raw = requests.get("https://movie.naver.com"+url) # 이를 새로운 res에 넣고 
        each_html = BeautifulSoup(each_raw.text, 'html.parser') # 영화페이지 각각을 파싱한 것을 each_html에 넣는다 

        html_list.append(each_html) # 이를 html_list에 넣는다 
        if len(html_list) == movienum:# len(html_list) = 영화의 개수만큼만 파싱
            break
            
    audience = [] # 관람객 평점
    reviewer = [] # 기자, 평론가 평점
    netizen = [] # 네티즌 평점 

    for each in html_list:
        scorelist_html = each.select("div.star_score em") # em으로 시작하는 값을 찾아서 

        scorelist = []    
        for score in scorelist_html: 
            scorelist.append(score.text) # 텍스트로 변환해 scorelist에 넣는다 

        audience.append("".join(scorelist[:4])) # 처음 네 글자가 관람객 평점 
        reviewer.append("".join(scorelist[4:8])) # 그 다음 네 글자가 기자, 평론가 평점 
        netizen.append("".join(scorelist[8:12])) # 그 다음의 네 글자가 네티즌 평점 

    return audience, reviewer, netizen 


### 3. 관람객 평점 공감순 20건 가져오기

평점, 평점 작성자 닉네임, 리뷰 본문

In [5]:
def get_reviews(movienum, reviewnum):
    url = 'https://movie.naver.com/movie/running/current.nhn'
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')    
    
    movie = soup.select("dl.lst_dsc")
    
    if reviewnum % 10 == 0: # review page의 page 수를 설정한다. 한 페이지 당 10개의 리뷰가 있으므로 
        page = (reviewnum//10) # 10으로 나누어떨어지면 그대로 몫으로 설정하고 
    else: 
        page = (reviewnum//10) + 1 # 10으로 나누어떨어지지 않는다면 몫+1을 해준다 (한계점: 강제로 올림해서 봐야 함...)
        
    
    codelist = []
    htmllist = []

    for m in movie :
        title = m.select_one("dt.tit a")  

        url = title.attrs["href"]

        # 영화 review page가 있는 url의 경우, 
        # https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=179181&type=after&isActualPointWriteExecute=false
        # &isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false 형태를 보인다
        # 이 뒤에 &page=n 형태가 붙는다 

        regex = re.compile(r'code=\d+') # url의 code=186821 부분을 추출해서
        urlcode = regex.search(url) 
        codelist.append(urlcode.group()) # codelist에 append 해준다 

    for i in range(movienum):
        eachhtmllist = []
        for j in range(page):
            each_raw = requests.get("https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?"+codelist[i]+"&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page="+str(j+1)) # 이를 새로운 res에 넣고 
            each_html = BeautifulSoup(each_raw.text, 'html.parser') # 영화 리뷰페이지 각각을 파싱한 것을 each_html에 넣는다 
            eachhtmllist.append(each_html)
        htmllist.append(eachhtmllist) # 이를 html_list에 넣는다 
        
    
    starlist = [] # 평점 
    nicknamelist = [] # 닉네임 
    reviewlist = [] # 리뷰 


    for movie in htmllist: 
        for html in movie: 
            reviewpage = html.select("div.score_result li")

            star = [] # 평점 
            nickname = [] # 닉네임 
            review = [] # 리뷰 

            for each in reviewpage: 
                star_i = each.select("div.star_score em")
                for s in star_i: 
                    star.append(s.text)

                nickname_i = each.select("div.score_reple dl dt em a span")
                for n in nickname_i: 
                    nickname.append(n.text)

                review_i = each.select("div.score_reple p")

                for rev in review_i: 
                    r = rev.text.strip()

                    r = re.sub('관람객[\n\t\r]+', '', r) # 관람객\n\n\r\n\t\t\t... 으로 시작된 단어 지우기 
                    r = re.sub('스포일러가 포함된 감상평입니다. 감상평 보기[\n\t\r]+', '', r) # 스포일러 ~ \n\r\n\t\t\t... 으로 시작된 단어 지우기

                    review.append(r)


            starlist.append(star)
            nicknamelist.append(nickname)
            reviewlist.append(review)


    # 리뷰 페이지마다 파싱했으므로, starlist를 출력하면 페이지마다의 평점이 나온다 
    # 하나의 영화에 대한 평점 리스트로 만들어준다         
    starlist_final = []
    for i in range(movienum):
        eachstar = []
        for j in range(page):
            eachstar.extend(starlist[i*page+j])
        starlist_final.append(eachstar)

    # 닉네임
    nicknamelist_final = []
    for i in range(movienum):
        eachnickname = []
        for j in range(page):
            eachnickname.extend(nicknamelist[i*page+j])
        nicknamelist_final.append(eachnickname)

    # 리뷰
    reviewlist_final = []
    for i in range(movienum):
        eachreview = []
        for j in range(page):
            eachreview.extend(reviewlist[i*page+j])
        reviewlist_final.append(eachreview)
        
    
    return starlist_final, nicknamelist_final, reviewlist_final
        

### 4. 저장하기

#### 1) 기본 정보

In [6]:
# 영화 정보 
def movieinfo(movienum, actornum): 
    titlelist, actorlist = movie_title_url_actor(movienum, actornum)
    audience, reviewer, netizen = get_grade(movienum)
    
    movie_info_dict = {"제목": titlelist, "주연배우": actorlist, 
               "관람객평점": audience, "기자/평론가평점": reviewer, "네티즌평점": netizen}
    
    return movie_info_dict

In [7]:
pd.DataFrame(movieinfo(5, 3))

,제목,주연배우,관람객평점,기자/평론가평점,네티즌평점
0,지푸라기라도 잡고 싶은 짐승들,"[전도연, 정우성, 배성우]",8.32,6.71,6.79
1,1917,"[조지 맥케이, 딘-찰스 채프먼]",9.35,7.67,8.98
2,인비저블맨,[엘리자베스 모스],7.67,7.83,7.67
3,정직한 후보,"[라미란, 김무열, 나문희]",8.60,5.38,7.71
4,작은 아씨들,"[시얼샤 로넌, 엠마 왓슨, 플로렌스 퓨]",9.19,8.00,8.88


In [8]:
# 리뷰 정보
def reviewinfo(movienum, reviewnum): 
    starlist_final, nicknamelist_final, reviewlist_final = get_reviews(movienum, reviewnum)
    
    starlist = []
    nicknamelist = []
    reviewlist = []
    
    for star, nick, review in zip(starlist_final, nicknamelist_final, reviewlist_final):
        for s,n,r in zip(star,nick,review):
            starlist.append(s)
            nicknamelist.append(n)
            reviewlist.append(r)
    
    review_info_dict = {"댓글평점": starlist, "닉네임": nicknamelist, "한줄평": reviewlist}
    
    return review_info_dict

In [9]:
pd.DataFrame(reviewinfo(5, 20))

,댓글평점,닉네임,한줄평
0,10,bohemian(mabu****),"난 전도연의 화류계 캐릭터가 좋다. 무뢰한, 너는 내 운명, 카운트다운...그리고 ..."
1,10,최정규(cjg4****),전도연 연기 진짜 오진다...와 이 영화에서 완전 섹시하게 나온다 역시 명불허전임...
2,10,달다(fxko****),8명의 배우가 모두 주인공 같은 느낌.
3,9,써니(tlag****),개존잼 역시 전도연이죠? 카리스마 미쳐벌여ㅠㅁㅠ
4,10,까칠소녀(oper****),"연출, 연기, 스토리 모두 대박...무조건 보세요."
...,...,...,...
95,8,rocker(nail****),시간의 재배치로 고전을 새롭게 만든 그레타 거윅의 마술. 뛰어난 감독이 손을 대니 ...
96,10,키엘(qjwq****),다들 왜이리 연기를 잘하는건가요..
97,10,선자(btid****),시얼샤로넌 연기 미쳤… 진짜 짱짱
98,10,하린(thee****),조역할을 한 배우님 연가 되게 잘하시더라


#### 2) 한꺼번에 보기

In [10]:
def moviecrawling(movienum, actornum, reviewnum): 
    titlelist, actorlist = movie_title_url_actor(movienum, actornum)
    audience, reviewer, netizen = get_grade(movienum)
    starlist_final, nicknamelist_final, reviewlist_final = get_reviews(movienum, reviewnum)
    
    # 영화 정보: dictionary 개수에 맞추기 위해서, 영화 리뷰 개수만큼 영화 정보 리스트 길이를 늘려준다 
    titles = []
    actors = []
    audiences = []
    reviewers = []
    netizens = []

    for i in range(movienum):
        titles[i*reviewnum:(i+1)*reviewnum] = [titlelist[i]]*reviewnum
        actors[i*reviewnum:(i+1)*reviewnum] = [actorlist[i]]*reviewnum
        audiences[i*reviewnum:(i+1)*reviewnum] = [audience[i]]*reviewnum
        reviewers[i*reviewnum:(i+1)*reviewnum] = [reviewer[i]]*reviewnum
        netizens[i*reviewnum:(i+1)*reviewnum] = [netizen[i]]*reviewnum
    
    
    # 리뷰 정보 
    starlist = []
    nicknamelist = []
    reviewlist = []
    
    for star, nick, review in zip(starlist_final, nicknamelist_final, reviewlist_final):
        for s,n,r in zip(star,nick,review):
            starlist.append(s)
            nicknamelist.append(n)
            reviewlist.append(r)

            
    movie_dict = {"제목": titles, "주연배우": actors, 
                   "관람객평점": audiences, "기자/평론가평점": reviewers, "네티즌평점": netizens,
                   "댓글평점": starlist, "닉네임": nicknamelist, "한줄평": reviewlist}
    

    return movie_dict   

In [11]:
movie = pd.DataFrame(moviecrawling(5, 3, 20))
movie

,제목,주연배우,관람객평점,기자/평론가평점,네티즌평점,댓글평점,닉네임,한줄평
0,지푸라기라도 잡고 싶은 짐승들,"[전도연, 정우성, 배성우]",8.32,6.71,6.79,10,bohemian(mabu****),"난 전도연의 화류계 캐릭터가 좋다. 무뢰한, 너는 내 운명, 카운트다운...그리고 ..."
1,지푸라기라도 잡고 싶은 짐승들,"[전도연, 정우성, 배성우]",8.32,6.71,6.79,10,최정규(cjg4****),전도연 연기 진짜 오진다...와 이 영화에서 완전 섹시하게 나온다 역시 명불허전임...
2,지푸라기라도 잡고 싶은 짐승들,"[전도연, 정우성, 배성우]",8.32,6.71,6.79,10,달다(fxko****),8명의 배우가 모두 주인공 같은 느낌.
3,지푸라기라도 잡고 싶은 짐승들,"[전도연, 정우성, 배성우]",8.32,6.71,6.79,9,써니(tlag****),개존잼 역시 전도연이죠? 카리스마 미쳐벌여ㅠㅁㅠ
4,지푸라기라도 잡고 싶은 짐승들,"[전도연, 정우성, 배성우]",8.32,6.71,6.79,10,까칠소녀(oper****),"연출, 연기, 스토리 모두 대박...무조건 보세요."
...,...,...,...,...,...,...,...,...
95,작은 아씨들,"[시얼샤 로넌, 엠마 왓슨, 플로렌스 퓨]",9.19,8.00,8.88,8,rocker(nail****),시간의 재배치로 고전을 새롭게 만든 그레타 거윅의 마술. 뛰어난 감독이 손을 대니 ...
96,작은 아씨들,"[시얼샤 로넌, 엠마 왓슨, 플로렌스 퓨]",9.19,8.00,8.88,10,키엘(qjwq****),다들 왜이리 연기를 잘하는건가요..
97,작은 아씨들,"[시얼샤 로넌, 엠마 왓슨, 플로렌스 퓨]",9.19,8.00,8.88,10,선자(btid****),시얼샤로넌 연기 미쳤… 진짜 짱짱
98,작은 아씨들,"[시얼샤 로넌, 엠마 왓슨, 플로렌스 퓨]",9.19,8.00,8.88,10,하린(thee****),조역할을 한 배우님 연가 되게 잘하시더라


#### 3) DataFrame 가공하기 

In [12]:
# 주연배우를 각각의 칼럼에 넣어준다
movie2 = pd.DataFrame.from_records(movie["주연배우"], columns=['주연배우1', '주연배우2', '주연배우3']) 

In [13]:
movie = pd.concat([movie, movie2], axis=1)
movie = pd.DataFrame(movie, columns=['제목', '주연배우1', '주연배우2', '주연배우3', 
                                      '관람객평점', '기자/평론가평점', '네티즌평점', 
                                      '댓글평점', '닉네임', '한줄평'])

In [14]:
movie

,제목,주연배우1,주연배우2,주연배우3,관람객평점,기자/평론가평점,네티즌평점,댓글평점,닉네임,한줄평
0,지푸라기라도 잡고 싶은 짐승들,전도연,정우성,배성우,8.32,6.71,6.79,10,bohemian(mabu****),"난 전도연의 화류계 캐릭터가 좋다. 무뢰한, 너는 내 운명, 카운트다운...그리고 ..."
1,지푸라기라도 잡고 싶은 짐승들,전도연,정우성,배성우,8.32,6.71,6.79,10,최정규(cjg4****),전도연 연기 진짜 오진다...와 이 영화에서 완전 섹시하게 나온다 역시 명불허전임...
2,지푸라기라도 잡고 싶은 짐승들,전도연,정우성,배성우,8.32,6.71,6.79,10,달다(fxko****),8명의 배우가 모두 주인공 같은 느낌.
3,지푸라기라도 잡고 싶은 짐승들,전도연,정우성,배성우,8.32,6.71,6.79,9,써니(tlag****),개존잼 역시 전도연이죠? 카리스마 미쳐벌여ㅠㅁㅠ
4,지푸라기라도 잡고 싶은 짐승들,전도연,정우성,배성우,8.32,6.71,6.79,10,까칠소녀(oper****),"연출, 연기, 스토리 모두 대박...무조건 보세요."
...,...,...,...,...,...,...,...,...,...,...
95,작은 아씨들,시얼샤 로넌,엠마 왓슨,플로렌스 퓨,9.19,8.00,8.88,8,rocker(nail****),시간의 재배치로 고전을 새롭게 만든 그레타 거윅의 마술. 뛰어난 감독이 손을 대니 ...
96,작은 아씨들,시얼샤 로넌,엠마 왓슨,플로렌스 퓨,9.19,8.00,8.88,10,키엘(qjwq****),다들 왜이리 연기를 잘하는건가요..
97,작은 아씨들,시얼샤 로넌,엠마 왓슨,플로렌스 퓨,9.19,8.00,8.88,10,선자(btid****),시얼샤로넌 연기 미쳤… 진짜 짱짱
98,작은 아씨들,시얼샤 로넌,엠마 왓슨,플로렌스 퓨,9.19,8.00,8.88,10,하린(thee****),조역할을 한 배우님 연가 되게 잘하시더라


#### 4) csv 파일로 저장하기 

In [15]:
movie.to_csv("movie.csv", index=False, encoding="utf-8-sig")